In [1]:
import pandas as pd

In [2]:
df_dicom = pd.read_csv("csv/dicom.csv")

/tmp/ipykernel_28667/149041353.py:1: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dicom = pd.read_csv("csv/dicom.csv")


In [3]:
len(df_dicom)

67749

In [4]:
df_dicom.value_counts(["VENDOR","CENTER"],sort=False)

VENDOR                      CENTER 
Agfa-Gevaert AG             asan           1
FUJI PHOTO FILM Co., ltd.   asan           2
FUJIFILM Corporation        asan           2
                            ncc2018        4
GE MEDICAL SYSTEMS          asan        3751
                            ncc2018     2191
                            ncc2020    13640
                            samsung     8693
                            twin        3685
HOLOGIC, Inc.               asan           4
                            brmh        5674
                            ilsan        402
                            ncc2018     6338
                            ncc2020    16360
                            samsung     1145
KODAK                       asan           1
KONICA MINOLTA              asan           1
LORAD                       asan           1
                            ncc2018        8
                            samsung     5808
                            twin          12
LUMISYS            

In [6]:
df_dicom.value_counts(["VENDOR"])

VENDOR                    
GE MEDICAL SYSTEMS            31960
HOLOGIC, Inc.                 29923
LORAD                          5829
SIEMENS                          19
FUJIFILM Corporation              6
MEDI-FUTURE MEDICAL SYSTEM        4
LUMISYS                           3
FUJI PHOTO FILM Co., ltd.         2
Agfa-Gevaert AG                   1
KODAK                             1
KONICA MINOLTA                    1
dtype: int64

In [5]:
%load_ext autoreload
%autoreload 2

import torch

import logging
import os
import sys
import tempfile
from glob import glob
import random
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
import cv2
import sys

import torch
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import ArrayDataset, create_test_image_2d, decollate_batch
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandRotate90,
    RandSpatialCrop,
    ScaleIntensity,
    NormalizeIntensity,
    #EnsureType,
    Resize
)
from monai.visualize import plot_2d_or_3d_image

import getfinal as f

/home/jslee/miniconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [6]:
# vendor 추출
df_image = pd.read_csv("csv/train.csv")
vendor_check = pd.DataFrame(index=range(0,len(df_image)), columns=['file','center', 'vendor'])

#result = {}
for i in tqdm(range(0,len(df_image))):
    file = 'dicom/'+df_image.CENTER[i]+'/'+ df_image.file[i]
    vendor_check.file[i] = df_image.file[i]
    vendor_check.center[i] = df_image.CENTER[i]
    vendor_check.vendor[i] = f.get_vendor(file)
    
vendor_check.to_csv("csv/vendor.csv",index=False)

vendor_check.value_counts(["center","vendor"])

100%|█████████████████████████████████████| 34118/34118 [36:10<00:00, 15.72it/s]


center   vendor
ncc2020  HO        8180
         GE        6782
asan     GE        3456
samsung  GE        3291
ncc2018  HO        2963
brmh     HO        2833
samsung  LORAD     2608
twin     GE        1790
ncc2018  GE         987
samsung  HO         898
ilsan    HO         295
asan     OTHERS      21
twin     LORAD        6
asan     HO           5
ncc2018  OTHERS       2
asan     LORAD        1
dtype: int64

In [8]:
vendor_check = pd.read_csv("csv/vendor.csv")
vendor_check.value_counts(["center","vendor"], sort=False)

center   vendor
asan     GE        3456
         HO           5
         LORAD        1
         OTHERS      21
brmh     HO        2833
ilsan    HO         295
ncc2018  GE         987
         HO        2963
         OTHERS       2
ncc2020  GE        6782
         HO        8180
samsung  GE        3291
         HO         898
         LORAD     2608
twin     GE        1790
         LORAD        6
dtype: int64

In [8]:
# vendor 붙이기
vendor_check = df_image.merge(vendor_check)
vendor_check

,dataset,dataset_density,dataset_image,file_image,file,CENTER,ID,DATE,SIDE,SIDE_case,GROUP,AGE,BMI,center,vendor
0,train,no,yes,numpy/image_256x256/asan/10512416_LCC_20070829...,10512416_LCC_20070829.dcm,asan,10512416,20070829.dcm,LCC,NaN,control,44.0,20.820,asan,GE
1,val,yes,yes,numpy/image_256x256/asan/10516254_LCC_20080709...,10516254_LCC_20080709.dcm,asan,10516254,20080709.dcm,LCC,RCC,case,48.0,23.740,asan,GE
2,train,no,yes,numpy/image_256x256/asan/10531695_LCC_20090205...,10531695_LCC_20090205.dcm,asan,10531695,20090205.dcm,LCC,NaN,control,55.0,21.841,asan,GE
3,train,yes,yes,numpy/image_256x256/asan/10531811_LCC_20080218...,10531811_LCC_20080218.dcm,asan,10531811,20080218.dcm,LCC,RCC,case,52.0,28.760,asan,GE
4,train,yes,yes,numpy/image_256x256/asan/10532852_LCC_20071214...,10532852_LCC_20071214.dcm,asan,10532852,20071214.dcm,LCC,NaN,control,49.0,19.453,asan,GE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34113,train,no,no,numpy/image_256x256/twin/E2004740_RCC_20071024...,E2004740_RCC_20071024.dcm,twin,E2004740,20071024.dcm,RCC,NaN,control,37.0,28.250,twin,GE
34114,train,no,no,numpy/image_256x256/twin/E2004750_RCC_20071024...,E2004750_RCC_20071024.dcm,twin,E2004750,20071024.dcm,RCC,NaN,control,37.0,24.410,twin,GE
34115,test,no,no,numpy/image_256x256/twin/E2004770_LCC_20071024...,E2004770_LCC_20071024.dcm,twin,E2004770,20071024.dcm,LCC,NaN,control,35.0,21.840,twin,GE
34116,train,no,no,numpy/image_256x256/twin/E2004780_RCC_20071024...,E2004780_RCC_20071024.dcm,twin,E2004780,20071024.dcm,RCC,NaN,control,35.0,23.130,twin,GE


In [9]:
# vendor가 OTHERS인 경우는 제외
list_others = vendor_check[vendor_check.vendor == "OTHERS"]
vendor = vendor_check[~vendor_check['file'].isin(list_others.file)]
vendor.value_counts(["center","vendor"])

center   vendor
ncc2020  HO        8180
         GE        6782
asan     GE        3456
samsung  GE        3291
ncc2018  HO        2963
brmh     HO        2833
samsung  LORAD     2608
twin     GE        1790
ncc2018  GE         987
samsung  HO         898
ilsan    HO         295
twin     LORAD        6
asan     HO           5
         LORAD        1
dtype: int64

In [11]:
list_others ## OTHERS 23개

,dataset,dataset_density,dataset_image,file_image,file,CENTER,ID,DATE,SIDE,SIDE_case,GROUP,AGE,BMI,center,vendor
1389,train,yes,yes,numpy/image_256x256/asan/34476206_RCC_20070119...,34476206_RCC_20070119.dcm,asan,34476206,20070119.dcm,RCC,LCC,case,47.0,21.79,asan,OTHERS
1393,train,yes,yes,numpy/image_256x256/asan/34557732_LCC_20070207...,34557732_LCC_20070207.dcm,asan,34557732,20070207.dcm,LCC,RCC,case,52.0,22.77,asan,OTHERS
1396,test,yes,yes,numpy/image_256x256/asan/34583344_RCC_20070214...,34583344_RCC_20070214.dcm,asan,34583344,20070214.dcm,RCC,LCC,case,52.0,20.93,asan,OTHERS
1403,train,yes,yes,numpy/image_256x256/asan/34666388_LCC_20070305...,34666388_LCC_20070305.dcm,asan,34666388,20070305.dcm,LCC,RCC,case,44.0,23.42,asan,OTHERS
1602,train,yes,yes,numpy/image_256x256/asan/36009190_LCC_20071229...,36009190_LCC_20071229.dcm,asan,36009190,20071229.dcm,LCC,RCC,case,50.0,24.34,asan,OTHERS
1611,train,yes,yes,numpy/image_256x256/asan/36069813_RCC_20070622...,36069813_RCC_20070622.dcm,asan,36069813,20070622.dcm,RCC,LCC,case,42.0,25.32,asan,OTHERS
2046,train,yes,yes,numpy/image_256x256/asan/37611510_RCC_20081217...,37611510_RCC_20081217.dcm,asan,37611510,20081217.dcm,RCC,LCC,case,40.0,24.67,asan,OTHERS
2067,cal,yes,yes,numpy/image_256x256/asan/38118773_LCC_20090404...,38118773_LCC_20090404.dcm,asan,38118773,20090404.dcm,LCC,RCC,case,42.0,27.94,asan,OTHERS
2072,train,yes,yes,numpy/image_256x256/asan/38350661_LCC_20090525...,38350661_LCC_20090525.dcm,asan,38350661,20090525.dcm,LCC,RCC,case,51.0,24.77,asan,OTHERS
2077,train,yes,yes,numpy/image_256x256/asan/38500992_LCC_20090615...,38500992_LCC_20090615.dcm,asan,38500992,20090615.dcm,LCC,RCC,case,55.0,27.19,asan,OTHERS


In [14]:
# check
len(df_image)-len(vendor)

23

In [10]:
vendor.to_csv("csv/train_vendor.csv",index=False)

--------------------------------------------

In [29]:
df_v = pd.read_csv("csv/train_vendor.csv")
df_n = pd.read_csv("csv/vendor.csv")

len(df_v)

34095

In [30]:
len(df_n)

34118

In [31]:
len(df_n)-len(df_v)

23

In [21]:
df = pd.read_csv("csv/train_vendor.csv")
df.value_counts(["dataset"], sort=False)

dataset  
NONE          8456
cal           2010
follow-up     5623
test          2008
train        13995
val           2003
dtype: int64

In [22]:
df_y = df[df.dataset_density=="yes"]
df_y.value_counts("dataset")

dataset
train    5646
cal       808
val       808
test      806
dtype: int64

In [26]:
df_y.value_counts("GROUP")

GROUP
control    5859
case       2209
dtype: int64

In [23]:
len(df_y)

8068

In [24]:
df_n = df[df.dataset_density == "no"]
df_n.value_counts("dataset")

dataset
NONE         8456
train        8349
follow-up    5623
cal          1202
test         1202
val          1195
dtype: int64

In [25]:
len(df_n)

26027

In [27]:
df_n.value_counts("GROUP")

GROUP
control    15203
case        4022
benign      2996
dtype: int64